notebook 7 
NBEATS
- wybór okna czasowego
- podzbiory kolumn
- optuna na strukturze
- sprawdzenie funkcji do metryk

In [1]:
pip install neuralforecast

INFO: pip is looking at multiple versions of neuralforecast to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 0.0/141.8 kB ? eta -:--:--
     ---------- -------------------------- 41.0/141.8 kB 991.0 kB/s eta 0:00:01
     ------------------------- ------------- 92.2/141.8 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 141.8/141.8 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/96.5 kB ? eta -:--:--
     ---------------- ----------------------- 41.0/96.5 kB 1.9 MB/s eta 0:00:01
     ---------------------------------------- 96.5/96.5 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/91.2 kB ? eta -:--:--
     ----------------- -------------------- 41.0/91.2 kB 991.0 kB/s eta 0:00:01
     ---------------------------------------- 91.2/91.2 kB 1.3 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of neuralforecast to det

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\HP\AppData\Local\Temp\pip-install-0x6mvtf8\numba_97f8cf0429ea4203a9f1bd52484ec2ca\setup.py", line 51, in <module>
          _guard_py_ver()
        File "C:\Users\HP\AppData\Local\Temp\pip-install-0x6mvtf8\numba_97f8cf0429ea4203a9f1bd52484ec2ca\setup.py", line 48, in _guard_py_ver
          raise RuntimeError(msg.format(cur_py, min_py, max_py))
      RuntimeError: Cannot install on Python version 3.12.0; only versions >=3.8,<3.12 are supported.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display, Markdown

import matplotlib.pyplot as plt
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS
from neuralforecast.utils import AirPassengersDF

# Split data and declare panel dataset
Y_df = AirPassengersDF
Y_train_df = Y_df[Y_df.ds<='1959-12-31'] # 132 train
Y_test_df = Y_df[Y_df.ds>'1959-12-31'] # 12 test

# Fit and predict with NBEATS and NHITS models
horizon = len(Y_test_df)
models = [NBEATS(input_size=2 * horizon, h=horizon, max_steps=50),
          NHITS(input_size=2 * horizon, h=horizon, max_steps=50)]
nf = NeuralForecast(models=models, freq='M')
nf.fit(df=Y_train_df)
Y_hat_df = nf.predict().reset_index()

# Plot predictions
fig, ax = plt.subplots(1, 1, figsize = (20, 7))
Y_hat_df = Y_test_df.merge(Y_hat_df, how='left', on=['unique_id', 'ds'])
plot_df = pd.concat([Y_train_df, Y_hat_df]).set_index('ds')

plot_df[['y', 'NBEATS', 'NHITS']].plot(ax=ax, linewidth=2)

ax.set_title('AirPassengers Forecast', fontsize=22)
ax.set_ylabel('Monthly Passengers', fontsize=20)
ax.set_xlabel('Timestamp [t]', fontsize=20)
ax.legend(prop={'size': 15})
ax.grid()